In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [17]:
%run -i ../../core.py

In [18]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [19]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.random.uniform(min(data), max(data), len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

# where(element > 30)
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
gt30_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 30].reshape(1,-1,1)
dummy_theory_output = torch.mean(gt30_dummy_data_tensor)



In [24]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.new_X = nn.Parameter(data = gt30_dummy_data_tensor,requires_grad = False )

    def forward(self,X):
        # where part
        # Many of these implementations are weird, but make it satisfy zkp of ezkl.
        len_ratio =  self.new_X.size()[1]/X.size()[1]
        X_where = torch.zeros(len_ratio*X.size()[1]).reshape(1,-1,1)
        X_where[0]=self.new_X[0]
        # constraint that new_X is indeed X where element is greater than 30
        new_X_cons = torch.sum((torch.abs(X[X>30].reshape(1,-1,1)[0]-X_where)<0.01*X_where).double())==X_where.size()[1]

        if new_X_cons:
            # value from mean calculation
            value = torch.abs(torch.sum(X_where)-X_where.size()[1]*(self.w))<=torch.abs(0.01*X_where.size()[1]*self.w)
        else:
            value = torch.tensor(0)

        return (value, self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_53805/4269092790.py:18: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if new_X_cons:
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/utils.py:1686: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you be

In [25]:
# prover calculates settings, send to verifier
gt30_data_tensor = data_tensor[data_tensor > 30].reshape(1,-1,1)
print("new tensor: ", gt30_data_tensor.size())
# gt30_data_tensor = torch.reshape(gt30_data_tensor_raw, (1, int(gt30_data_tensor_raw.size()[0]), 1))
theory_output = torch.mean(gt30_data_tensor)
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.new_X = nn.Parameter(data = gt30_data_tensor,requires_grad = False )
    def forward(self,X):
        # where part
        # Many of these implementations are weird, but make it satisfy zkp of ezkl.
        len_ratio =  self.new_X.size()[1]/X.size()[1]
        X_where = torch.zeros(len_ratio*X.size()[1]).reshape(1,-1,1)
        X_where[0]=self.new_X[0]
        # constraint that new_X is indeed X where element is greater than 30
        new_X_cons = torch.sum((torch.abs(X[X>30].reshape(1,-1,1)[0]-X_where)<0.01*X_where).double())==X_where.size()[1]

        if new_X_cons:
            # value from mean calculation
            value = torch.abs(torch.sum(X_where)-X_where.size()[1]*(self.w))<=torch.abs(0.01*X_where.size()[1]*self.w)
        else:
            value = torch.tensor(0)
        return (value, self.w)
    

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, [0], "resources", settings_path)

new tensor:  torch.Size([1, 272, 1])
Theory_output:  tensor(52.3676)
==== Generate & Calibrate Setting ====


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_53805/1512847886.py:22: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if new_X_cons:


RuntimeError: Failed to generate settings: Translating node #38 "/Sub" Sub ToTypedTranslator

In [22]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0


Time setup: 1.1649069786071777 seconds
Theory output:  tensor(52.3676)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 52.0
==== Generating Proof ====


spawning module 2


proof:  {'instances': [[[17970410297053518904, 9046703063145816218, 2851759239208196922, 164045840560226117], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [1460628720732602093, 15005283833837523956, 127539993950163949, 306168462079750959]]], 'proof': '28da2a9db1595c0a9050aadaa220a7bf9ff02418f2e4d229285b24b72f8b62832fc7902a62872cefcf353ca5d3aa169edbe479588b76e6deacbaf751ebeaf5652c83d8ba49277278ceb8181567e07065f006033550f26079e0bda50e1fd75c1a02d8207163fbca626ab0c32fe421797a2ff7218c6d8da58057322b24eb4c99c024e6062496c44ee9be4184ea17e75b1274ff8d01740668dfc193f81f369d5a830868033aef5f8c6b7ed413954b67da1b078e83d0e086b1189ed9115fbd1267872bf9bc9a130589448d058804366bdffc913c3a4d4c07f0d292d618ab73c6fcbc23310c0b17fef221309da7485f07528d23df8f822799c06425ee473c723568b01d81f4c2541cf7e031fb6da7367287dd83c4289445919696fe871ed45cdc800b015c3bc538e6b36a5592f2e379a7204c28d5ec355b7264cd48126eed0dcf3c9d1f17c0ffa916e8852ae205b37fed554ed3a4442b238646d5084fd1a8a1af855908

In [23]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[17970410297053518904, 9046703063145816218, 2851759239208196922, 164045840560226117], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [1460628720732602093, 15005283833837523956, 127539993950163949, 306168462079750959]]]
proof boolean:  1.0
proof result 1 : 52.0
verified
